## Decorators

In [1]:
def ff(f, a):
    return f(a)

In [2]:
ff(len, "12345")

5

In [3]:
def add_to(n):
    def adder(x):
        return n+x
    return adder

In [4]:
fa = add_to(6)

In [6]:
fa()

TypeError: adder() takes exactly 1 argument (0 given)

In [7]:
fa(11)

17

In [8]:
add_to(10)(7)

17

In [9]:
def string_required(f):
    def string_checked(arg):
        if type(arg) != str:
            raise ValueError(repr(arg)+" is not a string")
        return f(arg)
    return string_checked

In [10]:
# let's assume that we have a function, that must have string as an arg
def must_have_s(arg):
    print len(arg), arg

In [11]:
must_have_s("steve") # like that

5 steve


In [13]:
must_have_s([1, 2, 3]) # but list provides a valid output too

3 [1, 2, 3]


In [14]:
# so, we could do 
must_have_s = string_required(must_have_s)

In [15]:
must_have_s([1, 2, 3])

ValueError: [1, 2, 3] is not a string

In [16]:
# and syntax sugar
@string_required
def second(s):
    return 3*s

In [17]:
second("steve")

'stevestevesteve'

In [18]:
second((1, 2, 3))

ValueError: (1, 2, 3) is not a string

In [19]:
@string_required
class little:
    def __init__(self, s):
        self.s = s

In [20]:
person = little('steve')

In [21]:
person

<__main__.little instance at 0x7f7938b7fef0>

In [22]:
person2 = little([1, 2, 3])

ValueError: [1, 2, 3] is not a string

In [23]:
person.s

'steve'

In [24]:
def barking(cls):
    for name in cls.__dict__:
        if name.startswith("__"):
            continue
        func = getattr(cls, name) # func = cls.__dict__[name]
        def woofer(*args, **kwargs):
            print "Woof!"
            return func(*args, **kwargs)
        setattr(cls, name, woofer) # cls.__dict__[name] = woofer
    return cls

In [50]:
@barking
class dog_1:
    def shout(self):
        print "hello from dog 1"
    
    def fart(self):
        print "Ooops"

In [57]:
class dog_2:
    def shout(self):
        print "hello from dog 2"
    def fart(self):
        print "Ooops"

In [58]:
d1 = dog_1()

In [59]:
dir(d1)

['__doc__', '__module__', 'fart', 'shout']

In [60]:
d1.shout()

Woof!
hello from dog 1


In [61]:
dog_1.__dict__

{'__doc__': None,
 '__module__': '__main__',
 'fart': <function __main__.woofer>,
 'shout': <function __main__.woofer>}

In [62]:
dog_2.__dict__

{'__doc__': None,
 '__module__': '__main__',
 'fart': <function __main__.fart>,
 'shout': <function __main__.shout>}

In [29]:
@barking
class dog_3(dog_1):
    def wag(self):
        print "dog 3 is happy"
        
    def sniff(self):
        print "dog 3 is curious"

In [30]:
d3 = dog_3()

In [32]:
d3.wag(); d3.sniff(); d3.shout()

Woof!
dog 3 is curious
Woof!
dog 3 is curious
Woof!
hello from dog 1


In [33]:
def completely_barking(cls):
    class Null:
        pass
    return Null

In [34]:
@completely_barking
class something:
    def meth1(self):
        pass
    def meth2(self):
        pass
    

In [35]:
something

<class __main__.Null at 0x7f7938b91598>

In [36]:
something.meth1()

AttributeError: class Null has no attribute 'meth1'

In [5]:
# parameterized decorator

import time
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate


In [7]:
@clock() # clock, in fact, is a factory of decorators. Even without args we need parathesis to return decorator
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(i*0.125)


[0.00000572s] snooze(0.0) -> None
[0.12525868s] snooze(0.125) -> None
[0.25035143s] snooze(0.25) -> None


In [9]:
@clock('{name}: {elapsed:0.9f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(i*0.125)

snooze: 0.002908468s
snooze: 0.125305176s
snooze: 0.250334740s
